In [34]:
import torch
from torch import nn
from torch.autograd import Variable
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder
import torch.nn.functional as F
import torch.utils.data
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas as pd
import numpy as np
import csv
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [35]:
df = pd.read_csv('bank_data_nan_imputed.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355190 entries, 0 to 355189
Data columns (total 94 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0                   355190 non-null  int64  
 1   CR_PROD_CNT_IL               355190 non-null  int64  
 2   AMOUNT_RUB_CLO_PRC           355190 non-null  float64
 3   TURNOVER_DYNAMIC_IL_1M       355190 non-null  float64
 4   CNT_TRAN_AUT_TENDENCY1M      355190 non-null  float64
 5   SUM_TRAN_AUT_TENDENCY1M      355190 non-null  float64
 6   AMOUNT_RUB_SUP_PRC           355190 non-null  float64
 7   SUM_TRAN_AUT_TENDENCY3M      355190 non-null  float64
 8   CLNT_TRUST_RELATION          355190 non-null  int64  
 9   REST_DYNAMIC_FDEP_1M         355190 non-null  float64
 10  CNT_TRAN_AUT_TENDENCY3M      355190 non-null  float64
 11  APP_MARITAL_STATUS           355190 non-null  int64  
 12  REST_DYNAMIC_SAVE_3M         355190 non-null  float64
 13 

In [36]:
df.head()

,Unnamed: 0,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,SUM_TRAN_AUT_TENDENCY3M,CLNT_TRUST_RELATION,REST_DYNAMIC_FDEP_1M,...,LDEAL_USED_AMT_AVG_YQZ,REST_DYNAMIC_CC_1M,LDEAL_USED_AMT_AVG_YWZ,TURNOVER_DYNAMIC_CC_1M,AVG_PCT_DEBT_TO_DEAL_AMT,LDEAL_ACT_DAYS_ACC_PCT_AVG,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,TURNOVER_DYNAMIC_CC_3M,TARGET
0,0,0,0.000000,0.0,-1.000000,-1.000000,0.000000,-1.000000,6,0.0,...,-1.0,0.0,-1.0,0.0,-1.0,-1.0,0.0,-1.0,0.0,0
1,1,0,0.041033,0.0,0.166667,0.186107,0.244678,0.670968,6,0.0,...,-1.0,0.0,-1.0,0.0,-1.0,-1.0,0.0,-1.0,0.0,0
2,2,0,0.006915,0.0,-1.000000,-1.000000,0.000000,-1.000000,6,0.0,...,-1.0,0.0,-1.0,0.0,-1.0,-1.0,0.0,-1.0,0.0,0
3,3,0,0.000000,0.0,-1.000000,-1.000000,0.000000,-1.000000,6,0.0,...,-1.0,0.0,-1.0,0.0,-1.0,-1.0,0.0,-1.0,0.0,0
4,4,0,0.000000,0.0,-1.000000,-1.000000,0.000000,-1.000000,6,0.0,...,-1.0,0.0,-1.0,0.0,-1.0,-1.0,0.0,-1.0,0.0,0


In [6]:
df = df.astype(np.float64)

In [38]:
x_cols = df.columns.difference(['TARGET'])
X = df[x_cols]
y = df['TARGET']

In [39]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=113, shuffle=True, stratify=y)

In [40]:
from sklearn.preprocessing import StandardScaler
 
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test  = sc.transform(x_test)

In [43]:
x_train.shape

(284152, 93)

In [44]:
# Convert numpy array to tensor

# Train data
X_train = torch.from_numpy(x_train.astype(np.float32))
X_test = torch.from_numpy(x_test.astype(np.float32))
y_train = torch.from_numpy(y_train.values.astype(np.float32)).view(-1,1)
y_test = torch.from_numpy(y_test.values.astype(np.float32)).view(-1,1)

In [45]:
y_train = y_train.view(len(y_train))
y_test = y_test.view(len(y_test))

In [46]:
class ANN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 6)           # Input layer and Hidden layer1
        self.fc2 = nn.Linear(6,6)                    # Hidden layer1 and Hidden layer2
        self.output_layer = nn.Linear(6, output_dim) # Hidden layer2 and Output layer

    # feed forward function    
    def forward(self,x):
        x = F.relu(self.fc1(x))                      # Activation function for hidden layer1
        x = F.relu(self.fc2(x))                      # Activation function for hidden layer2
        x = F.sigmoid(self.output_layer(x))          # Activation function for output layer
        return x

In [47]:
model = ANN(input_dim = x_train.shape[1], output_dim = 1)

print(model)

ANN(
  (fc1): Linear(in_features=93, out_features=6, bias=True)
  (fc2): Linear(in_features=6, out_features=6, bias=True)
  (output_layer): Linear(in_features=6, out_features=1, bias=True)
)


In [48]:
# Loss function (Binary Cross Entropy loss)
criterion = nn.BCELoss()

# Optimizer  
optimizer = optim.Adam(model.parameters(), lr = 0.01)

In [49]:
data_test   = Variable(X_test).float()
target_test = Variable(y_test).type(torch.FloatTensor)

# predict test data
y_pred_test = model(data_test)
before_train = criterion(y_pred_test.squeeze(), target_test.squeeze())

# Print the loss before training the test data
print('Test loss before training' , before_train.item())

Test loss before training 0.630792498588562


C:\Users\acer\anaconda3\envs\ML\lib\site-packages\torch\nn\functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [50]:
model.train()

for epoch in range(1000):
    data = Variable(X_train).float()
    target = Variable(y_train).type(torch.FloatTensor)
    
    # forward pass
    output = model(data)
    loss   = criterion(output.squeeze(), target)
    
    # Backward pass (set grad to zero, pass the loss backward, apply optimizer and update weights)
    optimizer.zero_grad() # Sets gradients to zero for each iteration
    loss.backward()       # Perform backward pass to compute gradients
    optimizer.step()      # Update weights 
    
    if (epoch+1) % 10 == 0:
        print ('epoch [{}/{}], Loss: {:.3f}'.format(epoch+1, 1000, loss.item()))

epoch [10/1000], Loss: 0.485
epoch [20/1000], Loss: 0.360
epoch [30/1000], Loss: 0.291
epoch [40/1000], Loss: 0.276
epoch [50/1000], Loss: 0.268
epoch [60/1000], Loss: 0.266
epoch [70/1000], Loss: 0.263
epoch [80/1000], Loss: 0.262
epoch [90/1000], Loss: 0.260
epoch [100/1000], Loss: 0.259
epoch [110/1000], Loss: 0.259
epoch [120/1000], Loss: 0.258
epoch [130/1000], Loss: 0.257
epoch [140/1000], Loss: 0.257
epoch [150/1000], Loss: 0.256
epoch [160/1000], Loss: 0.255
epoch [170/1000], Loss: 0.254
epoch [180/1000], Loss: 0.254
epoch [190/1000], Loss: 0.253
epoch [200/1000], Loss: 0.252
epoch [210/1000], Loss: 0.251
epoch [220/1000], Loss: 0.251
epoch [230/1000], Loss: 0.250
epoch [240/1000], Loss: 0.250
epoch [250/1000], Loss: 0.250
epoch [260/1000], Loss: 0.249
epoch [270/1000], Loss: 0.249
epoch [280/1000], Loss: 0.248
epoch [290/1000], Loss: 0.248
epoch [300/1000], Loss: 0.247
epoch [310/1000], Loss: 0.247
epoch [320/1000], Loss: 0.246
epoch [330/1000], Loss: 0.246
epoch [340/1000], L

In [51]:
# Calculate the test loss after updating the weights

data_test   = Variable(X_test).float()
target_test = Variable(y_test).type(torch.FloatTensor)

# predict test data
y_pred_test = model(data_test)
after_train = criterion(y_pred_test.squeeze(), target_test.squeeze())

# Print the loss after training the test data
print('Test loss after training' , after_train.item())

Test loss after training 0.2296106368303299


In [52]:
metrics.roc_auc_score(y_test, y_pred_test.squeeze().detach().numpy())

0.8185663895735344